In [ ]:
# 导入必要的库
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
from timm.layers import trunc_normal_, DropPath
from torchvision.datasets import CIFAR100
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm  

def set_seed(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True


set_seed(42)

# 创建保存目录
save_dir = './pretrained_checkpoints'
os.makedirs(save_dir, exist_ok=True)

# 设置曲线保存路径
curve_path = os.path.join(save_dir, 'training_curves.png')

In [ ]:
class LayerNorm(nn.Module):
  
    def __init__(self, normalized_shape, eps=1e-6, data_format="channels_last"):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(normalized_shape))
        self.bias = nn.Parameter(torch.zeros(normalized_shape))
        self.eps = eps
        self.data_format = data_format
        if self.data_format not in ["channels_last", "channels_first"]:
            raise NotImplementedError 
        self.normalized_shape = (normalized_shape, )
    
    def forward(self, x):
        if self.data_format == "channels_last":
            return F.layer_norm(x, self.normalized_shape, self.weight, self.bias, self.eps)
        elif self.data_format == "channels_first":
            u = x.mean(1, keepdim=True)
            s = (x - u).pow(2).mean(1, keepdim=True)
            x = (x - u) / torch.sqrt(s + self.eps)
            x = self.weight[:, None, None] * x + self.bias[:, None, None]
            return x

class Block(nn.Module):
    """ConvNeXt Block"""
    def __init__(self, dim, drop_path=0., layer_scale_init_value=1e-6):
        super().__init__()
        self.dwconv = nn.Conv2d(dim, dim, kernel_size=7, padding=3, groups=dim) 
        self.norm = LayerNorm(dim, eps=1e-6)
        self.pwconv1 = nn.Linear(dim, 4 * dim) 
        self.act = nn.GELU()
        self.pwconv2 = nn.Linear(4 * dim, dim)
        self.gamma = nn.Parameter(layer_scale_init_value * torch.ones((dim)), 
                                    requires_grad=True) if layer_scale_init_value > 0 else None
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()

    def forward(self, x):
        input = x
        x = self.dwconv(x)
        x = x.permute(0, 2, 3, 1) # (N, C, H, W) -> (N, H, W, C)
        x = self.norm(x)
        x = self.pwconv1(x)
        x = self.act(x)
        x = self.pwconv2(x)
        if self.gamma is not None:
            x = self.gamma * x
        x = x.permute(0, 3, 1, 2) # (N, H, W, C) -> (N, C, H, W)

        x = input + self.drop_path(x)
        return x

In [ ]:
class ConvNeXt(nn.Module):
    """ConvNeXt"""
    def __init__(self, in_chans=3, num_classes=100, 
                 depths=[3, 3, 9, 3], dims=[96, 192, 384, 768], drop_path_rate=0.1, 
                 layer_scale_init_value=1e-6, head_init_scale=1.):
        super().__init__()

        # 下采样层
        self.downsample_layers = nn.ModuleList()
        stem = nn.Sequential(
            nn.Conv2d(in_chans, dims[0], kernel_size=4, stride=4),
            LayerNorm(dims[0], eps=1e-6, data_format="channels_first")
        )
        self.downsample_layers.append(stem)
        for i in range(3):
            downsample_layer = nn.Sequential(
                LayerNorm(dims[i], eps=1e-6, data_format="channels_first"),
                nn.Conv2d(dims[i], dims[i+1], kernel_size=2, stride=2),
            )
            self.downsample_layers.append(downsample_layer)

        # 主干网络阶段
        self.stages = nn.ModuleList()
        dp_rates=[x.item() for x in torch.linspace(0, drop_path_rate, sum(depths))] 
        cur = 0
        for i in range(4):
            stage = nn.Sequential(
                *[Block(dim=dims[i], drop_path=dp_rates[cur + j], 
                layer_scale_init_value=layer_scale_init_value) for j in range(depths[i])]
            )
            self.stages.append(stage)
            cur += depths[i]

        # 分类头
        self.norm = nn.LayerNorm(dims[-1], eps=1e-6)
        self.head = nn.Linear(dims[-1], num_classes)

        # 权重初始化
        self.apply(self._init_weights)
        self.head.weight.data.mul_(head_init_scale)
        self.head.bias.data.mul_(head_init_scale)

    def _init_weights(self, m):
        if isinstance(m, (nn.Conv2d, nn.Linear)):
            trunc_normal_(m.weight, std=.02)
            nn.init.constant_(m.bias, 0)

    def forward_features(self, x):
        for i in range(4):
            x = self.downsample_layers[i](x)
            x = self.stages[i](x)
        return self.norm(x.mean([-2, -1])) # 全局平均池化

    def forward(self, x):
        x = self.forward_features(x)
        x = self.head(x)
        return x

In [ ]:
# 预训练模型URL
model_urls = {
    "convnext_tiny_1k": "https://dl.fbaipublicfiles.com/convnext/convnext_tiny_1k_224_ema.pth",
    "convnext_small_1k": "https://dl.fbaipublicfiles.com/convnext/convnext_small_1k_224_ema.pth",
    "convnext_base_1k": "https://dl.fbaipublicfiles.com/convnext/convnext_base_1k_224_ema.pth",
    "convnext_large_1k": "https://dl.fbaipublicfiles.com/convnext/convnext_large_1k_224_ema.pth",
}

def convnext_tiny_cifar100(pretrained=True):
    """创建预训练的ConvNeXt Tiny模型"""
    model = ConvNeXt(depths=[3, 3, 9, 3], dims=[96, 192, 384, 768], num_classes=100)
    
    if pretrained:
        # 加载预训练权重
        url = model_urls['convnext_tiny_1k']
        checkpoint = torch.hub.load_state_dict_from_url(url=url, map_location="cpu", check_hash=True)
        
        # 移除分类头部分（因为CIFAR100是100类，而ImageNet是1000类）
        if 'model' in checkpoint:
            checkpoint = checkpoint['model']
        if 'head.weight' in checkpoint:
            checkpoint.pop('head.weight')
        if 'head.bias' in checkpoint:
            checkpoint.pop('head.bias')
        
        # 加载预训练权重
        model.load_state_dict(checkpoint, strict=False)
        print("预训练权重加载完成！")
    
    return model

In [5]:
def get_cifar100_loaders(batch_size=64, num_workers=2):
    """数据加载和预处理 - 去掉数据增强"""
    # 简单的数据转换，无数据增强
    transform = transforms.Compose([
        transforms.Resize(224),  # ConvNeXt期望224x224输入
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5071, 0.4867, 0.4408], std=[0.2675, 0.2565, 0.2761])
    ])
    
    # 加载CIFAR100数据集
    train_dataset = CIFAR100(root='./data', train=True, download=True, transform=transform)
    valid_dataset = CIFAR100(root='./data', train=False, download=True, transform=transform)
    
    # 创建数据加载器
    train_loader = DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True,
        num_workers=num_workers, pin_memory=True
    )
    
    valid_loader = DataLoader(
        valid_dataset, batch_size=batch_size, shuffle=False,
        num_workers=num_workers, pin_memory=True
    )
    
    return train_loader, valid_loader

# 获取数据加载器
train_loader, valid_loader = get_cifar100_loaders(batch_size=64)
print(f"数据集大小 - 训练集: {len(train_loader.dataset)}, 验证集: {len(valid_loader.dataset)}")

  1%|          | 2.03M/169M [00:34<46:58, 59.3kB/s]  


KeyboardInterrupt: 

In [ ]:
def train_one_epoch(model, train_loader, criterion, optimizer, device):
    """训练一个epoch"""
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    
    pbar = tqdm(train_loader, desc='训练中')
    for batch_idx, (inputs, targets) in enumerate(pbar):
        inputs, targets = inputs.to(device), targets.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        pbar.set_postfix({
            'loss': train_loss/(batch_idx+1),
            'acc': 100.*correct/total
        })
    
    return train_loss/len(train_loader), 100.*correct/total

def validate(model, valid_loader, criterion, device):
    """验证函数"""
    model.eval()
    valid_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        pbar = tqdm(valid_loader, desc='验证中')
        for batch_idx, (inputs, targets) in enumerate(pbar):
            inputs, targets = inputs.to(device), targets.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            valid_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            
            pbar.set_postfix({
                'loss': valid_loss/(batch_idx+1),
                'acc': 100.*correct/total
            })
    
    return valid_loss/len(valid_loader), 100.*correct/total

def plot_training_curves(train_losses, valid_losses, train_accs, valid_accs, save_path=curve_path):
    """绘制训练曲线并保存"""
    plt.figure(figsize=(14, 5))
    
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='训练损失')
    plt.plot(valid_losses, label='验证损失')
    plt.xlabel('轮次')
    plt.ylabel('损失')
    plt.legend()
    plt.title('损失曲线')
    
    plt.subplot(1, 2, 2)
    plt.plot(train_accs, label='训练准确率')
    plt.plot(valid_accs, label='验证准确率')
    plt.xlabel('轮次')
    plt.ylabel('准确率 (%)')
    plt.legend()
    plt.title('准确率曲线')
    
    plt.tight_layout()
    
    # 保存图像
    plt.savefig(save_path)
    
    # 在Jupyter中显示图像
    plt.show()

In [ ]:
# 设置训练参数
lr = 1e-5                 # 学习率
batch_size = 64           # 批次大小  
num_epochs = 10           # 训练轮次
weight_decay = 1e-4       # 权重衰减
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"使用设备: {device}")

# 创建模型（预训练）
model = convnext_tiny_cifar100(pretrained=True)
model = model.to(device)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
scheduler = CosineAnnealingLR(optimizer, T_max=num_epochs)

In [ ]:
# 训练循环
train_losses, valid_losses = [], []
train_accs, valid_accs = [], []
best_acc = 0

print(f"开始预训练模型微调，总共{num_epochs}轮...")

for epoch in range(num_epochs):
    print(f"\n轮次: {epoch+1}/{num_epochs}")
    
    # 训练
    train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    
    # 验证
    valid_loss, valid_acc = validate(model, valid_loader, criterion, device)
    valid_losses.append(valid_loss)
    valid_accs.append(valid_acc)
    
    # 更新学习率
    scheduler.step()
    
    # 保存最佳模型
    if valid_acc > best_acc:
        best_acc = valid_acc
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'accuracy': valid_acc,
        }, os.path.join(save_dir, 'best_model.pth'))
        print(f"保存最佳模型，准确率: {valid_acc:.2f}%")
    
    # 绘制训练曲线
    plot_training_curves(train_losses, valid_losses, train_accs, valid_accs, curve_path)

print(f"训练完成! 最佳验证准确率: {best_acc:.2f}%")

In [ ]:
# 加载最佳模型测试
best_model = convnext_tiny_cifar100(pretrained=False)
checkpoint = torch.load(os.path.join(save_dir, 'best_model.pth'), map_location=device)
best_model.load_state_dict(checkpoint['model_state_dict'])
best_model = best_model.to(device)

# 测试最佳模型
test_loss, test_acc = validate(best_model, valid_loader, criterion, device)
print(f"最佳模型在测试集上的准确率: {test_acc:.2f}%")